In [2]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rasterio.features
import rioxarray as rio
import xrspatial.multispectral as ms
import pandas as pd
# Import Planetary Computer tools
import pystac_client
import planetary_computer as pc
import odc
from odc.stac import stac_load
from odc.algo import to_rgba
from tqdm import tqdm
tqdm.pandas()

In [3]:
#read_csv
df = pd.read_csv('challenge_1_submission_template_correct_columns_fixed.csv')

In [5]:
lat_longs = list(df['id'].apply(lambda x: x[1:-1]))
#lat_longs = list(df['id'].apply(lambda x: x[1:-1]))
box_size_deg = 0.05
time_window="2021-11-01/2022-08-31"
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")


In [6]:
# Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees
resolution =  10 # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for CRS:4326 

In [7]:
# To mask the pixels and find clouds or water, it is best to use the bit values of the 16-bit qa_pixel flag
# See the website above for a nice explanation of the process

bit_flags = {
            'fill': 1<<0,
            'dilated_cloud': 1<<1,
            'cirrus': 1<<2, 
            'cloud': 1<<3,
            'shadow': 1<<4, 
            'snow': 1<<5, 
            'clear': 1<<6,
            'water': 1<<7
}

In [8]:
# Create a function that will mask pixels with a given type

def get_mask(mask, flags_list):
    
    # Create the result mask filled with zeros and the same shape as the mask
    final_mask = np.zeros_like(mask)
    
    # Loop through the flags  
    for flag in flags_list:
        
        # get the mask for each flag
        flag_mask = np.bitwise_and(mask, bit_flags[flag])
        
        # add it to the final flag
        final_mask = final_mask | flag_mask
    
    return final_mask > 0

In [47]:
from tqdm import tqdm
#define column
Latitude_Longtitudes = list()
time_frames = list()
original_red = list()
original_blue= list()
original_green = list()
original_ndvi = list()

filter_red = list()
filter_blue= list()
filter_green = list()
filter_ndvi = list()

filter_lir = list()
filter_swir= list()
original_lir= list()
original_swir = list()

original_nir = list()
filter_nir= list()
original_savi= list()
filter_savi= list()
original_arvi = list()
filter_arvi= list()
original_evi = list()
filter_evi= list()

In [48]:
for lat_long in tqdm(lat_longs[:100]):
    lat_long = tuple(map(float, lat_long.split(', ')))
    min_lon = lat_long[1]-box_size_deg/2
    min_lat = lat_long[0]-box_size_deg/2
    max_lon = lat_long[1]+box_size_deg/2
    max_lat = lat_long[0]+box_size_deg/2
    bounds = (min_lon, min_lat, max_lon, max_lat)
    
    search = stac.search(
    collections=["landsat-c2-l2"],  #Landsat Collection 2 Level-2  #https://planetarycomputer.microsoft.com/dataset/group/landsat
    bbox=bounds, 
    datetime=time_window,
    query={"platform": {"in": ["landsat-8", "landsat-9"]},},
    )
    
    items = search.get_all_items()
    
    # https://odc-stac.readthedocs.io/en/latest/notebooks/stac-load-S2-ms.html#Configuration
    xx = stac_load(
    items,
    #bands=["red", "green", "blue", "nir08", "qa_pixel"],  
    bands=["red", "green", "blue", "nir08", "qa_pixel", "swir16", "lwir11"],  
            #bands to calculate nvdi: "red", "nir08"
            #bands for adjust brightness of images: "red", "green", "blue"
            #bands for filtering mask:  "qa_pixel", "qa_radsat", "qa_aerosol"
    crs="EPSG:4326", # Latitude-Longitude
    resolution=scale, # Degrees
    chunks={"x": 2048, "y": 2048},
    patch_url=pc.sign,
    bbox=bounds
    )
    
    # Apply scaling and offsets for Landsat Collection-2 (reference below) to the spectral bands ONLY
    # https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2
    xx['red'] = (xx['red']*0.0000275)-0.2
    xx['green'] = (xx['green']*0.0000275)-0.2
    xx['blue'] = (xx['blue']*0.0000275)-0.2
    xx['nir08'] = (xx['nir08']*0.0000275)-0.2
    
    if (len(xx.time) == 59) or (len(xx.time) == 58) or (len(xx.time) == 57):
        xx = xx.drop_isel(time=[len(xx.time) - 10])
    if len(xx.time) == 31:
        xx = xx.drop_isel(time=[25])
    if (len(xx.time) == 63) or (len(xx.time) == 64) or (len(xx.time) == 65):
        xx = xx.drop_isel(time=[50,51,52])
    if len(xx.time) == 91:
        xx = xx.drop_isel(time=[74,75])
    if (len(xx.time) == 66) or (len(xx.time) == 67):
        xx = xx.drop_isel(time=[53,54])
        
    full_mask = get_mask(xx['qa_pixel'], ['fill', 'dilated_cloud', 'cirrus', 'cloud', 'shadow', 'water'])
    cleaned_data = xx.where(~full_mask)
    
    # Calculate the mean of the data across the sample region and then NDVI
    # Perform this calculation for the unfiltered and cloud-filtered (clean) datasets
    time_frame = xx.time.values
    
    mean_unfiltered = xx.mean(dim=['longitude','latitude']).compute()
    mean_red = mean_unfiltered.red.values
    mean_blue = mean_unfiltered.blue.values
    mean_green = mean_unfiltered.green.values
    mean_nir = mean_unfiltered.nir08.values
    ndvi_mean = ((mean_unfiltered.nir08-mean_unfiltered.red)/(mean_unfiltered.nir08+mean_unfiltered.red)).values
    mean_swir = mean_unfiltered.swir16.values
    mean_lwir = mean_unfiltered.lwir11.values
    # mean_savi = ((mean_unfiltered.nir08 - mean_unfiltered.red)/(mean_unfiltered.nir08+mean_unfiltered.red +0.5)) * (1 + 0.5).values
    # mean_arvi = (mean_unfiltered.nir08 - (mean_unfiltered.red -1*(mean_unfiltered.blue - mean_unfiltered.red)))/(mean_unfiltered.nir08 + (mean_unfiltered.red -(mean_unfiltered.blue - mean_unfiltered.red))).values
    # mean_evi = 2.5*((mean_unfiltered.nir08 - mean_unfiltered.red)/((mean_unfiltered.nir08 + 6*mean_unfiltered.red -7.5*mean_unfiltered.blue)+1)).values
   
    mean_clean = cleaned_data.mean(dim=['longitude','latitude']).compute()
    mean_red_clean = mean_clean.red.values
    mean_blue_clean = mean_clean.blue.values
    mean_green_clean = mean_clean.green.values
    mean_nir_clean = mean_clean.nir08.values
    ndvi_mean_clean = ((mean_clean.nir08-mean_clean.red)/(mean_clean.nir08+mean_clean.red)).values
    mean_swir_clean = mean_clean.swir16.values
    mean_lwir_clean = mean_clean.lwir11.values
    # mean_savi_clean = ((mean_clean.nir08 - mean_clean.red)/(mean_clean.nir08+mean_clean.red +0.5)) * (1 + 0.5).values
    # mean_arvi_clean = (mean_clean.nir08 - (mean_clean.red -1*(mean_clean.blue - mean_clean.red)))/(mean_clean.nir08 + (mean_clean.red - (mean_clean.blue - mean_clean.red))).values
    # mean_evi_clean = 2.5*((mean_clean.nir08 - mean_clean.red)/((mean_clean.nir08 + 6*mean_clean.red -7.5*mean_clean.blue)+1)).values

    
    Latitude_Longtitudes.extend([lat_long]*len(time_frame))
    time_frames.extend(time_frame)
    original_red.extend(mean_red)
    original_blue.extend(mean_blue)
    original_green.extend(mean_green)
    original_ndvi.extend(ndvi_mean)
    original_lir.extend(mean_lwir)
    original_swir.extend(mean_swir)
    original_nir.extend(mean_nir)
    # original_savi.extend(mean_savi)
    # original_arvi.extend(mean_arvi)
    # original_evi.extend(mean_evi)
    
    
    
    filter_red.extend(mean_red_clean)
    filter_blue.extend(mean_blue_clean)
    filter_green.extend(mean_green_clean)
    filter_ndvi.extend(ndvi_mean_clean)
    filter_lir.extend(mean_lwir_clean)
    filter_swir.extend(mean_swir_clean)
    filter_nir.extend(mean_nir_clean)
    # filter_savi.extend(mean_savi_clean)
    # filter_arvi.extend(mean_arvi_clean)
    # filter_evi.extend(mean_evi_clean)

100%|██████████| 100/100 [56:13<00:00, 33.74s/it] 


In [52]:
df_new = pd.DataFrame({'Latitude_Longtitudes': Latitude_Longtitudes,
        'time_frames': time_frames,
        'original_red':original_red,
        'original_blue':original_blue,
        'original_green': original_green,
        'original_ndvi' :original_ndvi,
        'filter_red':filter_red,
        'filter_blue':filter_blue,
        'filter_green':filter_green ,
        'filter_ndvi': filter_ndvi,
        'filter_lir': filter_lir,
        'original_lir': original_lir,
        'original_swir': original_swir,
        'filter_swir': filter_swir,
        'original_nir': original_nir,
        'filter_nir': filter_nir})

In [53]:
df_new['time_frames'] = df_new['time_frames'].dt.strftime('%d/%m/%Y')

In [55]:
df_new['original_savi'] = ((df_new['original_nir'] - df_new['original_red'])/(df_new['original_nir']+df_new['original_red'] +0.5)) * (1 + 0.5)
df_new['filter_savi'] = ((df_new['filter_nir'] - df_new['filter_red'])/(df_new['filter_nir']+df_new['filter_red'] +0.5)) * (1 + 0.5)
df_new['original_arvi'] = (df_new['original_nir'] - (df_new['original_red'] -1*(df_new['original_blue'] - df_new['original_red'])))/(df_new['original_nir'] + (df_new['original_red'] -(df_new['original_blue'] - df_new['original_red'])))
df_new['filter_arvi'] = (df_new['filter_nir'] - (df_new['filter_red'] -1*(df_new['filter_blue'] - df_new['filter_red'])))/(df_new['filter_nir'] + (df_new['filter_red'] -(df_new['filter_blue'] - df_new['filter_red'])))

df_new['original_evi'] = 2.5*((df_new['original_nir'] - df_new['original_red'])/((df_new['original_nir'] + 6*df_new['original_red'] -7.5*df_new['original_blue'])+1))
df_new['filter_evi'] = 2.5*((df_new['filter_nir'] - df_new['filter_red'])/((df_new['filter_nir'] + 6*df_new['filter_red'] -7.5*df_new['filter_blue'])+1))


In [ ]:
df_new.to_csv("train-landsat.csv", index=False)